# 04. 보안 (Security)

## 학습 목표

이 노트북에서는 Istio의 **보안(Security)** 기능, 특히 **Authorization Policy**를 학습합니다.

**테스트 시나리오:**
1. 기본 상태 - 모든 트래픽 허용
2. DENY 정책 - 특정 서비스 차단
3. ALLOW 정책 - 특정 서비스만 허용
4. 경로/메서드별 세분화된 접근 제어

**학습 포인트:**
- Zero Trust 보안 모델
- 서비스 간 접근 제어를 코드 없이 구현
- 최소 권한 원칙 적용

---

## 사전 조건 체크

In [ ]:
# 환경 확인
!kubectl get pods -n istio-demo | grep -E "httpbin|sleep"

---

## 개념 설명: Zero Trust 보안

### 전통적 보안 vs Zero Trust

**전통적 보안 (Castle and Moat):**
- 외부는 위험, 내부는 안전
- 방화벽 안쪽은 신뢰
- 문제: 내부 침해 시 확산 용이

**Zero Trust:**
- "신뢰하지 않고, 항상 검증"
- 모든 요청은 인증/인가 필요
- 최소 권한 원칙

### Istio Authorization Policy

```yaml
apiVersion: security.istio.io/v1
kind: AuthorizationPolicy
metadata:
  name: httpbin-policy
spec:
  selector:
    matchLabels:
      app: httpbin  # 대상 서비스
  action: ALLOW  # 또는 DENY
  rules:
    - from:
        - source:
            principals: ["cluster.local/ns/istio-demo/sa/sleep"]
      to:
        - operation:
            methods: ["GET"]
            paths: ["/get", "/headers"]
```

---

## 시나리오 1: 기본 상태 - 모든 트래픽 허용

Istio의 기본 설정은 **모든 서비스 간 통신을 허용**합니다.

In [ ]:
# 기본 상태에서 sleep → httpbin 요청 (성공 예상)
print("기본 상태: sleep → httpbin 요청")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code} (200 예상)\n" http://httpbin:8000/get

**결과:** HTTP 200 - 정책이 없으면 모든 요청 허용

---

## 시나리오 2: DENY 정책 - 모든 트래픽 차단

**Zero Trust의 시작점**: 모든 접근을 차단한 후, 필요한 것만 허용

```yaml
apiVersion: security.istio.io/v1
kind: AuthorizationPolicy
metadata:
  name: deny-all
spec:
  selector:
    matchLabels:
      app: httpbin
  {}  # 빈 spec = 모든 요청 거부
```

In [ ]:
# Deny All 정책 적용
!kubectl apply -f ../03-security/authorization-policy-deny-all.yaml

In [ ]:
# 정책 적용 대기
import time
print("정책 적용 대기 (2초)...")
time.sleep(2)

In [ ]:
# sleep → httpbin 요청 (차단 예상)
print("Deny All 적용 후: sleep → httpbin 요청")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code} (403 예상)\n" http://httpbin:8000/get

**결과:** HTTP 403 Forbidden - "RBAC: access denied"

In [ ]:
# Deny All 정책 삭제
!kubectl delete -f ../03-security/authorization-policy-deny-all.yaml --ignore-not-found
import time
time.sleep(1)

---

## 시나리오 3: ALLOW 정책 - 특정 서비스만 허용

**sleep** 서비스에서만 **httpbin**에 접근 가능하도록 설정합니다.

### Service Account 기반 식별

Istio는 Kubernetes **Service Account**를 사용하여 서비스를 식별합니다.

```yaml
principals:
  - "cluster.local/ns/istio-demo/sa/sleep"
```

- `cluster.local`: 클러스터 도메인
- `ns/istio-demo`: 네임스페이스
- `sa/sleep`: Service Account 이름

In [ ]:
# Allow Sleep 정책 적용
!kubectl apply -f ../03-security/authorization-policy-allow-sleep.yaml

In [ ]:
# 정책 적용 대기
import time
print("정책 적용 대기 (2초)...")
time.sleep(2)

In [ ]:
# sleep → httpbin 요청 (허용 예상)
print("Allow Sleep 적용 후: sleep → httpbin 요청")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code} (200 예상)\n" http://httpbin:8000/get

**결과:** HTTP 200 - sleep 서비스는 허용됨

In [ ]:
# Allow Sleep 정책 삭제
!kubectl delete -f ../03-security/authorization-policy-allow-sleep.yaml --ignore-not-found
import time
time.sleep(1)

---

## 시나리오 4: 경로/메서드별 세분화된 접근 제어

더 세밀한 접근 제어를 적용합니다:
- **GET /get, /headers**: 허용
- **POST /post**: 차단
- **/status/***:  차단

In [ ]:
# Path-based 정책 적용
!kubectl apply -f ../03-security/authorization-policy-path-based.yaml

In [ ]:
# 정책 적용 대기
import time
print("정책 적용 대기 (2초)...")
time.sleep(2)

In [ ]:
# 테스트 1: GET /get (허용)
print("테스트 1: GET /get (허용 예상)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code}\n" http://httpbin:8000/get

In [ ]:
# 테스트 2: GET /headers (허용)
print("테스트 2: GET /headers (허용 예상)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code}\n" http://httpbin:8000/headers

In [ ]:
# 테스트 3: POST /post (차단)
print("테스트 3: POST /post (403 차단 예상)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code}\n" -X POST http://httpbin:8000/post

In [ ]:
# 테스트 4: GET /status/200 (차단)
print("테스트 4: GET /status/200 (403 차단 예상)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code}\n" http://httpbin:8000/status/200

**결과 요약:**

| 요청 | 예상 결과 | 이유 |
|------|----------|------|
| GET /get | 200 | 허용된 경로 |
| GET /headers | 200 | 허용된 경로 |
| POST /post | 403 | POST 메서드 차단 |
| GET /status/200 | 403 | /status/* 경로 차단 |

In [ ]:
# Path-based 정책 삭제
!kubectl delete -f ../03-security/authorization-policy-path-based.yaml --ignore-not-found
print("정리 완료!")

---

## 보안 테스트 완료!

### 핵심 정리

| 정책 타입 | 동작 | 사용 사례 |
|----------|------|----------|
| Deny All | 모든 요청 차단 | Zero Trust 시작점 |
| Allow 특정 소스 | 지정 서비스만 허용 | 서비스 간 접근 제어 |
| Path/Method 기반 | 경로/메서드별 세분화 | API 레벨 접근 제어 |

**Istio 보안 장점:**
- 코드 수정 없이 보안 정책 적용
- Service Account 기반 강력한 식별
- 런타임에 정책 변경 가능
- mTLS로 통신 암호화 (기본 활성화)

### 다음 단계

**[05-게이트웨이.ipynb](./05-게이트웨이.ipynb)** - Kubernetes Gateway API를 활용한 외부 트래픽 관리를 학습합니다.